## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

1. Подключим все необходимое

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pymorphy2


     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 7.7 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import nltk 
import string

from sklearn.metrics import * 
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from pymorphy2 import MorphAnalyzer

from tqdm import tqdm

nltk.download('stopwords')
nltk.download('punkt')
data = pd.read_csv('/content/drive/MyDrive/data/women-clothing-accessories.csv', sep='\t', usecols=[0,1])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2. Лемматизируем датасет и реализуем функцию составления отчета по результатам оценок

In [ ]:
def lemmatize(x_data):
  pymrth_analyzer = MorphAnalyzer()
  sentences = []
  
  for j in range(len(x_data)):
    sentence = x_data[j]

    for ch in string.punctuation:
      sentence = sentence.replace(ch,"")

    sent = word_tokenize(sentence)
    normal_sent = ""
    
    for i in range(len(sent)):
      word = pymrth_analyzer.parse(sent[i])
      normal_word = word[0].normal_form
      normal_sent += (normal_word + " ") 

    sentences.append(normal_sent)

  return pd.Series(sentences)
def make_report_df(report, vectorizer_name='no_name'):
  comment_types = ['neautral', 'negative', 'positive']
  score_types = ['precision', 'recall', 'f1-score']

  result = {'accuracy': report['accuracy']}

  for score in score_types:
    avg = 0
    for comment in comment_types:
      avg = avg + report[comment][score]
    avg = avg / len(comment_types)
    result[score] = avg

  return pd.DataFrame(data=result, index=[vectorizer_name])
global_report_df = pd.DataFrame()

In [ ]:
lemmatized_data_review = lemmatize(data.review)
lemmatized_data_review

0        качество плохой пошив ужасный горловина напере...
1        товар отдать другой человек я не получить посы...
2        ужасный синтетик тонкий ничего общий с предста...
3        товар не прийти продавец продлить защита без м...
4              кофточка голый синтетик носить не возможно 
                               ...                        
89995    сделать достаточно хорошо на ткань сделать рис...
89996    накидка шикарный спасибо большой провдо линять...
89997    спасибо большой продовца рекомендовать заказат...
89998    очень довольный заказ маленький месяц в рб кур...
89999    хороший куртка посторонний запах нет шов ровны...
Length: 90000, dtype: object

3. Разделим предобработанные данные

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(lemmatized_data_review, data.sentiment, train_size = 0.7)
noise = stopwords.words('russian')

4. Запустим подбор параметров для мешка n-грамм

In [ ]:
pipeline = Pipeline([
           ('vect', CountVectorizer(tokenizer=word_tokenize, stop_words=noise)),
           ('clf', MultinomialNB()),
])
parameters = [{
    'vect__ngram_range': ((1,2), (1,3), (2,2), (2,3), (3,3))  
}]

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, cv=2)
grid_search.fit(X_train, y_train)
best_n, best_m = grid_search.best_params_['vect__ngram_range']

print("( ", best_n, " , ", best_m, " ) : best parameters for n-gramms bag")

In [ ]:
best_n, best_m = 1, 2

5. Применим их

In [ ]:
vectorizer = CountVectorizer(tokenizer=word_tokenize, ngram_range=(best_n, best_m), stop_words=noise)
vectorized_x_train = vectorizer.fit_transform(X_train)
model = MultinomialNB()
model.fit(vectorized_x_train, y_train)
vectorized_x_test = vectorizer.transform(X_test)

In [ ]:
pred = model.predict(vectorized_x_test)
vect_compare = classification_report(y_test, pred, output_dict=True)

6. Запустим подбор параметров для tf-idf

In [ ]:
pipeline_1 = Pipeline([
           ('vect', TfidfVectorizer(tokenizer=word_tokenize, stop_words=noise)),
           ('clf', MultinomialNB()),
])
parameters_1 = [{
    'vect__ngram_range': ((1,2), (1,3), (2,2), (2,3), (3,3)),
    'vect__max_df': [0.2 , 0.4, 0.6, 0.8, 1.0],
    'vect__min_df': [0.  , 0.2, 0.4, 0.6, 0.8, 1.0],
    'vect__max_features': [50000, 100000, 150000]
}]

grid_search_Tfidf = GridSearchCV(pipeline_1, parameters_1, n_jobs=-1, cv=2, verbose=10)
grid_search_Tfidf.fit(X_train, y_train)


Fitting 2 folds for each of 450 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   53.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed: 15.7min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed: 18

GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        no

In [ ]:
best_n, best_m = grid_search_Tfidf.best_params_['vect__ngram_range']
best_max_df = grid_search_Tfidf.best_params_['vect__max_df']
best_min_df = grid_search_Tfidf.best_params_['vect__min_df']
best_max_features = grid_search_Tfidf.best_params_['vect__max_features']

print("( ", best_n, " , ", best_m, " ) : best parameters for Tfidf")
print(best_max_df, " ", best_min_df, " ", best_max_features, " : best max_df, min_df and max_features for Tfidf")

(  1  ,  2  ) : best parameters for Tfidf
0.2   0.0   150000  : best max_df, min_df and max_features for Tfidf


Значения параметра max_features больше 150_000 вызывают переобучение, а меньше - модель недообучается

In [ ]:
best_n, best_m, best_max_df, best_min_df, best_max_features = 1, 2, 0.2, 0.0, 150000

7. Применим их

In [ ]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(best_n, best_m), tokenizer=word_tokenize, stop_words=noise, max_df=best_max_df, min_df=best_min_df, max_features=best_max_features)
tfidf_vectorized_x_train = tfidf_vectorizer.fit_transform(X_train)

clf = MultinomialNB()
clf.fit(tfidf_vectorized_x_train, y_train)

tfidf_vectorized_x_test = tfidf_vectorizer.transform(X_test)
pred = clf.predict(tfidf_vectorized_x_test)
report_tfidf = classification_report(y_test, pred, output_dict=True)

8. Запустим подбор параметров для символьных n-грамм

In [ ]:
pipeline = Pipeline([
           ('vect', CountVectorizer(analyzer='char')),
           ('clf', MultinomialNB()),
])
parameters = [{
    'vect__ngram_range': ((3,6), (3,8), (3,10), (4,8), (4,10), (5,9), (5,11), (6,12))  
}]

grid_search_char = GridSearchCV(pipeline, parameters, n_jobs=-1, cv=2, verbose=10)
grid_search_char.fit(X_train, y_train)
best_n, best_m = grid_search_char.best_params_['vect__ngram_range']
best_n, best_m

print("( ", best_n, " , ", best_m, " ) : best parameters for char vectorizer")

Fitting 2 folds for each of 8 candidates, totalling 16 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   51.8s
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed: 14.3min finished


(  5  ,  11  ) : best parameters for char vectorizer


In [ ]:
best_n, best_m = 5, 11

9. Применим их

In [ ]:
char_vectorizer = CountVectorizer(ngram_range=(best_n, best_m), analyzer='char')
char_vectorized_x_train = char_vectorizer.fit_transform(X_train)

clf = MultinomialNB()
clf.fit(char_vectorized_x_train, y_train)
char_vectorized_x_test = char_vectorizer.transform(X_test)
pred = clf.predict(char_vectorized_x_test)
report_char_ngram = classification_report(y_test, pred, output_dict=True)

Составим таблицу усредненных по количеству sentiment оценок

In [ ]:
global_report_df = pd.DataFrame()
global_report_df = global_report_df.append(make_report_df(vect_compare, "n-gramms bag"))
global_report_df = global_report_df.append(make_report_df(report_tfidf, "tf-idf"))
global_report_df = global_report_df.append(make_report_df(report_char_ngram, "char n-gram"))
global_report_df

,accuracy,precision,recall,f1-score
n-gramms bag,0.710788,0.711183,0.711672,0.710905
tf-idf,0.711196,0.712609,0.712023,0.711462
char n-gram,0.733639,0.738406,0.734321,0.735465


Составим полную таблицу оценок по всем sentiment

In [ ]:
import itertools
global_report_df = pd.DataFrame()
vect_compare = pd.DataFrame().from_dict(dict(itertools.islice(vect_compare.items(), 4)))
vect_compare['name'] = 'n-gramm bag'
global_report_df = global_report_df.append(vect_compare[:-1])
vect_compare = pd.DataFrame().from_dict(dict(itertools.islice(report_tfidf.items(), 4)))
vect_compare['name'] = 'tf-idf'
global_report_df = global_report_df.append(vect_compare[:-1])
vect_compare = pd.DataFrame().from_dict(dict(itertools.islice(report_char_ngram.items(), 4)))
vect_compare['name'] = 'char'
global_report_df = global_report_df.append(vect_compare[:-1])
global_report_df

,neautral,negative,positive,accuracy,name
precision,0.602708,0.702955,0.827753,0.710788,n-gramm bag
recall,0.629405,0.654232,0.847273,0.710788,n-gramm bag
f1-score,0.615767,0.677719,0.837399,0.710788,n-gramm bag
precision,0.600479,0.704904,0.831453,0.710974,tf-idf
recall,0.637026,0.649611,0.844738,0.710974,tf-idf
f1-score,0.618213,0.676129,0.838043,0.710974,tf-idf
precision,0.615270,0.720882,0.881179,0.733899,char
recall,0.683641,0.659867,0.856419,0.733899,char
f1-score,0.647656,0.689026,0.868623,0.733899,char


Исходя из полученных оценок можно сказать что лучший результат показал векторайзер символьные n-граммы. TF-IDF и мешок n-грамм сработали примерно одинаково.

## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?
Потому что сказано что findall возвращает список всех непересекающихся совпадений, а abcx пересекается с abca

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
stroka = 'Ваня вышел погулять, вновь увидел самокат'
res = re.findall(r'\b\w{1,2}', stroka) 
print(res)

['Ва', 'вы', 'по', 'вн', 'ув', 'са']


### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
stroka = 'itsy. bitsy. teenie. weenie.'
res = re.split(r'\.+', stroka, maxsplit=2)
print(res)

['itsy', ' bitsy', ' teenie. weenie.']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
stroka = "22*2=4 это база"
res = re.sub(r'[0-9]', 'DIG', stroka)
print(res)

DIGDIG*DIG=DIG это база


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
stroka = "ссылка на видео https://www.youtube.com/watch?v=i0tfiaIXcBE"
res = re.sub(r'(s|ht|f)(m?)(tp?)(s?):\S+', '', stroka)
print(res)

ссылка на видео 


### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[\w\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile(r'[\b\w-]{4,}')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'больше', 'больше', 'слов', 'Что-то']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
prog = re.compile(r'@\w+.\w+')
prog.findall('abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz')

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']